## 1. PP-LCNetv2 模型简介
PP-LCNetv2 系列模型是 PaddleCV 团队提出的轻量级卷积神经网络模型，该系列模型基于 PP-LCNetv1 改进而来，更为侧重较高的模型性能，在 ImageNet1k 数据集上可以达到 77.04% 的精度，同时推理延时在 4.32ms 左右。另外，考虑到产业界大量使用 OpenVINO 推理框架的情况，PP-LCNetv2 系列模型针对 Intel CPU 硬件平台结合 OpenVINO 推理框架特别优化。

PP-LCNetv2 系列模型基于 PaddleClas 开发、训练，更多关于 PP-LCNetv2 系列模型的更多信息，可以从 [PaddleClas-PPLCNetv2](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/zh_CN/models/ImageNet1k/PP-LCNetV2.md) 获取。

## 2. 模型效果

PP-LCNetv2 系列模型在 ImageNet1k 数据集上的评测指标如下所示：

| Model | Params(M) | FLOPs(M) | Top-1 Acc(\%) | Top-5 Acc(\%) | Latency(ms) |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| <b>PPLCNetv2_base<b>  | <b>6.6<b> | <b>604<b>  | <b>77.04<b> | <b>93.27<b> | <b>4.32<b> |
| <b>PPLCNetv2_base_ssld<b>  | <b>6.6<b> | <b>604<b>  | <b>80.07<b> | <b>94.87<b> | <b>4.32<b> |

其中 `_ssld` 表示使用 `SSLD 蒸馏`后的模型。关于 `SSLD蒸馏` 的内容，详情 [SSLD 蒸馏](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/zh_CN/training/advanced/knowledge_distillation.md)。

与同量级其他轻量级网络的性能对比：

| Model | Params(M) | FLOPs(M) | Top-1 Acc(\%) | Top-5 Acc(\%) | Latency(ms) |
|:--:|:--:|:--:|:--:|:--:|:--:|
| MobileNetV3_Large_x1_25 | 7.4 | 714  | 76.4 | 93.00 | 5.19 |
| PPLCNetv1_x2_5  | 9 | 906  | 76.60 | 93.00 | 7.25 |
| <b>PPLCNetv2_base<b>  | <b>6.6<b> | <b>604<b>  | <b>77.04<b> | <b>93.27<b> | <b>4.32<b> |
| <b>PPLCNetv2_base_ssld<b>  | <b>6.6<b> | <b>604<b>  | <b>80.07<b> | <b>94.87<b> | <b>4.32<b> |

## 3. 模型快速使用

### 3.1 模型推理：
* 安装相关 Python 包

（不在Jupyter Notebook上运行时需要将"!"去掉。）


如果您的机器安装了 CUDA9、CUDA10 或 CUDA11，请运行以下命令安装 paddle

In [ ]:
!pip install paddlepaddle-gpu

您的机器是CPU，请运行以下命令安装 paddle

In [ ]:
!pip install paddlepaddle

安装 paddleclas

In [ ]:
!pip install paddleclas

* 快速体验

恭喜！ 您已经成功安装了 PaddleClas，接下来快速体验图像分类效果。

In [ ]:
paddleclas --model_name=PPLCNetv2_base  --infer_imgs="docs/images/inference_deployment/whl_demo.jpg"

上述命令的运行结果如下所示：

class_ids: [8, 7, 86, 82, 83], scores: [0.8859, 0.07156, 0.00588, 0.00047, 0.00034], label_names: ['hen', 'cock', 'partridge', 'ruffed grouse, partridge, Bonasa umbellus', 'prairie chicken, prairie grouse, prairie fowl'], filename: docs/images/inference_deployment/whl_demo.jpg
Predict complete

### 3.2 模型训练
* PP-LCNetv2 系列模型基于 PaddleClas 实现，模型训练的具体信息请参考[模型训练、评估和预测](https://github.com/PaddlePaddle/PaddleClas/blob/release/2.5/docs/zh_CN/models/ImageNet1k/PP-LCNetV2.md#3-%E6%A8%A1%E5%9E%8B%E8%AE%AD%E7%BB%83%E8%AF%84%E4%BC%B0%E5%92%8C%E9%A2%84%E6%B5%8B)。

## 4. 模型原理

PP-LCNetv2 网络的整体结构如下图所示：

<div align="center">
<img src="https://user-images.githubusercontent.com/12560511/200776415-19f7f09d-f8e9-4b64-bbcd-72ac418c23cf.png"  width = "80%"  />
</div>

## 5. 注意事项
PP-LCNetv2 系列模型在 Intel CPU 端延时测试基于 Intel Xeon Gold 6271C 硬件平台完成，推理框架使用 OpenVINO 2021.4.2，推理 batch size 为 1，线程数为 10。

## 6. 相关论文以及引用信息